Bru's notebook

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import sys
sys.path.append('../../utils')
import functions

In [ ]:
clean_df = pd.read_csv('../../data/raw/Clean_Dataset.csv')
business_df = pd.read_csv('../../data/raw/business.csv')
economy_df = pd.read_csv('../../data/raw/economy.csv')

In [ ]:
functions.show_basic_info(clean_df)

In [ ]:
functions.show_basic_info(business_df)

In [ ]:
functions.show_basic_info(economy_df)

# Data Cleaning

## Rename Columns

In [ ]:
clean_df.drop(columns=['Unnamed: 0'], inplace=True) # drop duplicate index and days left columns

In [ ]:
replacement_dict = {
    'airline': 'airline_name',
    'flight':'flight_code',
    'source_city': 'departure_city',
    'duration': 'flight_duration',
    'destination_city': 'arrival_city',
    'duration':'flight_duration',
    'price': 'price'
}

clean_df = functions.rename_columns(clean_df, replacement_dict)
clean_df.head()

In [ ]:
replacement_dict = {
    'date': 'flight_date',
    'airline': 'airline_name',
    'ch_code': 'airline_code',
    'num_code': 'flight_number',
    'dep_time': 'departure_time',
    'from': 'departure_city',
    'time_taken': 'flight_duration',
    'stop': 'stops',
    'arr_time': 'arrival_time',
    'to': 'arrival_city',
    'price': 'price'
}

business_df = functions.rename_columns(business_df, replacement_dict)
business_df.head()

In [ ]:
replacement_dict = {
    'date': 'flight_date',
    'airline': 'airline_name',
    'ch_code': 'airline_code',
    'num_code': 'flight_number',
    'dep_time': 'departure_time',
    'from': 'departure_city',
    'time_taken': 'flight_duration',
    'stop': 'stops',
    'arr_time': 'arrival_time',
    'to': 'arrival_city',
    'price': 'price'
}

economy_df = functions.rename_columns(economy_df, replacement_dict)
economy_df.head()

## Remove duplicate rows

In [ ]:
functions.remove_duplicates(clean_df)

In [ ]:
functions.remove_duplicates(business_df)

In [ ]:
functions.remove_duplicates(economy_df)

## Data Standardization
- Ensuring matching data types
- Formatting values consistently
- Normalizing textual data

## Standardize 'price' column
Convert data type from string to integer in business and economy dataframes

In [ ]:
def standardize_price(df, column_name='price'):
    df[column_name] = df[column_name].str.replace(',', '', regex=True).astype(int)
    return df

business_df = standardize_price(business_df, 'price')
economy_df = standardize_price(economy_df, 'price')

print('Business DF:')
print(business_df)
print('\nEconomy DF:')
print(economy_df)

## Standardize 'stops' column
economy and business dataframes have unclear values in the 'stops' column, parse them to integers (0, 1 or 2) 

In [ ]:
# clean_df
stop_mapping = {
        'zero': 0,
        'one': 1,
        'two_or_more': 2
    }
    
clean_df['stops'] = clean_df['stops'].map(stop_mapping)

In [ ]:
def stops_to_integers(df, column_name='stops'):
    def map_stops(value):
        if 'non-stop' in value:
            return 0
        else:
            match = re.search(r'\d+', value) # extract first digit w regex
            return int(match.group()) if match else None

    df[column_name] = df[column_name].apply(map_stops)
    return df

In [ ]:
stops_to_integers(business_df)

In [ ]:
stops_to_integers(economy_df)

## Date & Time Conversion
- flight_date columns
- departure_time
- arrival_time

TBD: decide how to handle departure and arrival times in cleaned_df

In [ ]:
functions.convert_date_column(economy_df, 'flight_date')

In [ ]:
functions.convert_date_column(business_df, 'flight_date')

In [ ]:
functions.convert_time_column(business_df, 'arrival_time')
functions.convert_time_column(business_df, 'departure_time')

In [ ]:
functions.convert_time_column(economy_df, 'arrival_time')
functions.convert_time_column(economy_df, 'departure_time')

## Generate Flight Code
combine 'airline_code' and 'flight_number' columns to generate 'flight_code' - business and economy dataframes, to match clean df

In [ ]:
def create_flight_code(df):
    df['flight_code'] = df['airline_code'] + '-' + df['flight_number'].astype(str)
    return df

In [ ]:
create_flight_code(business_df)

In [ ]:
create_flight_code(economy_df)

#### Drop columns

In [ ]:
business_df = business_df.drop(columns=['airline_code', 'flight_number'])
economy_df = economy_df.drop(columns=['airline_code', 'flight_number'])

## Generate Flight Time Groups
- business_df and economy_df have datetime information
- clean_df has time categories (Early morning, morning, etc.)
- Create group columns and assign time group according to predefined range

In [ ]:
# function to generate time groups
def categorize_flight_time(time_str):
    hour = int(time_str.split(':')[0]) 
    
    # categorize based on the hour block
    if 4 <= hour < 6:
        return 'Early Morning'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 21:
        return 'Evening'
    elif 21 <= hour < 24:
        return 'Night'
    else:
        return 'Late Night'
    

# implementation

business_df['departure_time_group'] = business_df['departure_time'].apply(categorize_flight_time)
business_df['arrival_time_group'] = business_df['arrival_time'].apply(categorize_flight_time)

economy_df['departure_time_group'] = economy_df['departure_time'].apply(categorize_flight_time)
economy_df['arrival_time_group'] = economy_df['arrival_time'].apply(categorize_flight_time)

# clean_df
clean_df.rename(columns={'departure_time': 'departure_time_group', 'arrival_time' : 'arrival_time_group'}, inplace=True)

columns_to_clean = ['departure_time_group', 'arrival_time_group']

for col in columns_to_clean:
    clean_df[col] = clean_df[col].str.replace(' ', '', regex=True).str.replace('_', ' ', regex=True)

## Standardize Airline Names
For congruence in every dataframe

In [ ]:
# mapping function for standardization
def standardize_airline_names(df, column_name='airline_name'):
    standard_name_mapping = {
        'SpiceJet': 'SpiceJet',
        'AirAsia': 'Air Asia',
        'Vistara': 'Vistara',
        'GO_FIRST': 'Go First',
        'GO FIRST': 'Go First',
        'Indigo': 'Indigo',
        'Air_India': 'Air India',
        'Air India': 'Air India',
        'Trujet': 'TruJet',
        'StarAir': 'Star Air',
    }
    
    df[column_name] = df[column_name].replace(standard_name_mapping)
    return df

# implementation
clean_df = standardize_airline_names(clean_df)
print(clean_df['airline_name'].unique())

business_df = standardize_airline_names(business_df)
print(clean_df['airline_name'].unique())

economy_df = standardize_airline_names(economy_df)
print(economy_df['airline_name'].unique())

In [ ]:
clean_df.head()

## Format 'flight_duration' column
- Convert duration to minutes(int)
- Separate function for economy and business dfs, handling the different format and dot notation indicating dates.

In [ ]:
def convert_duration_to_minutes(df, column_name):
    df[column_name] = (df[column_name] * 60).round().astype(int)
    return df

clean_df = convert_duration_to_minutes(clean_df, 'flight_duration')

In [ ]:
def convert_duration_string_to_minutes(duration_str):
    hours, minutes = 0, 0
    
    if '1.' in duration_str: # extract days
        hours = 24 
        duration_str = duration_str.replace('1.', '')

    if 'h' in duration_str: # extract hours
        hours += int(duration_str.split('h')[0])

    if 'm' in duration_str: # extract minutes
        try:
            minutes_str = duration_str.split('h ')[1].split('m')[0]  # Extract minutes portion after 'h'
            minutes = int(minutes_str) if minutes_str else 0  # If minutes_str is empty, set minutes to 0
        except IndexError:
            minutes = 0  # if no value after 'h'
    
    return int(hours * 60 + minutes) # return total in minutes


business_df['flight_duration'] = business_df['flight_duration'].apply(convert_duration_string_to_minutes)

In [ ]:
economy_df['flight_duration'] = economy_df['flight_duration'].apply(convert_duration_string_to_minutes)

#### Reorder columns

In [ ]:
def reorder_columns(df, column_order):
    df = df[column_order]
    return df

column_order = [
    'flight_date', 'airline_name', 'flight_code', 
    'departure_time', 'departure_city', 
    'arrival_time', 'arrival_city', 
    'flight_duration', 'stops', 'price', 
    'departure_time_group', 'arrival_time_group'
]

In [ ]:
business_df = reorder_columns(business_df, column_order)

In [ ]:
economy_df = reorder_columns(economy_df, column_order)

In [ ]:
clean_df = clean_df[[
    'airline_name', 'flight_code', 
    'departure_city', 
    'arrival_city', 
    'flight_duration', 'stops', 'price', 'class', 'days_left',
    'departure_time_group', 'arrival_time_group'
]]

### Check again

In [ ]:
clean_df.head()

In [ ]:
business_df.head()

In [ ]:
economy_df.head()

### Export Data Frames to .csv

In [ ]:
# clean_df.to_csv('../../data/clean/clean_dataset_df.csv', index=False)
# business_df.to_csv('../../data/clean/clean_business_df.csv', index=False)
# economy_df.to_csv('../../data/clean/clean_economy_df.csv', index=False)